In [2]:
import os, sys, multiprocessing as mp, pandas as pd, requests, re
from bs4 import BeautifulSoup
pd.set_option('max_colwidth', 200)

In [2]:
r = requests.get('https://deckstats.net/decks/99839/957744-muldrotha-edh?lng=en')
r.content
soup = BeautifulSoup(r.content, 'lxml')

html_data = soup.find_all('div', {'id': 'deck_overview_cards'})

raw_data = str(html_data[0])
indices = [m.start() for m in re.finditer('target="_blank">', raw_data)]
card_list = []
for index in indices:
    card_text = raw_data[index+16:index+56]
    card_text = card_text[0:card_text.find(' </a>')].strip()
    card_list.append(card_text)
len(card_list) # len should be lower than total (duplicate basic lands)

178

In [4]:
cached_data = 'muldrotha.csv'
def get_card_data(card_name):
    r = requests.get('https://api.scryfall.com/cards/search/?q=' + card_name)
    return(r.json()) # we _should_ always get the card we need since titles are exact
card_data = []
if not os.path.exists(cached_data):
    pool = mp.Pool()
    card_data = pool.map(get_card_data, card_list)
    pool.close()

In [23]:
card_df = None
if os.path.exists(cached_data):
    card_df =pd.read_csv('muldrotha.csv')
else:
    card_df = pd.DataFrame(list(map(lambda x: x['data'][0], card_data)))
    card_df = card_df[['name','type_line','cmc','mana_cost','oracle_text','power','toughness','usd']]
    card_df.to_csv('muldrotha.csv', sep = ',', encoding = 'utf-8', index = False)
card_df.head()

,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
0,"Muldrotha, the Gravetide",Legendary Creature — Elemental Avatar,6.0,{3}{U}{B}{G},"During each of your turns, you may play up to one permanent card of each permanent type from your graveyard. (If a card has multiple permanent types, choose one as you play it.)",6,6,6.49
1,Abrupt Decay,Instant,2.0,{B}{G},Abrupt Decay can't be countered by spells or abilities.\nDestroy target nonland permanent with converted mana cost 3 or less.,NaN,NaN,4.69
2,Acidic Slime,Creature — Ooze,5.0,{3}{G}{G},"Deathtouch (Any amount of damage this deals to a creature is enough to destroy it.)\nWhen Acidic Slime enters the battlefield, destroy target artifact, enchantment, or land.",2,2,0.25
3,Alchemist's Refuge,Land,0.0,NaN,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: You may cast spells this turn as though they had flash.",NaN,NaN,2.06
4,Arcane Lighthouse,Land,0.0,NaN,"{T}: Add {C} to your mana pool.\n{1}, {T}: Until end of turn, creatures your opponents control lose hexproof and shroud and can't have hexproof or shroud.",NaN,NaN,2.79


In [17]:
lands = card_df[card_df['type_line'].str.contains('Land', na = False)]
print('Land count: ' + str(len(lands)))
lands[['name','type_line','oracle_text','usd']].sort_values('usd', ascending=False)

Land count: 49


,name,type_line,oracle_text,usd
38,Dark Depths,Legendary Snow Land,"Dark Depths enters the battlefield with ten ice counters on it.\n{3}: Remove an ice counter from Dark Depths.\nWhen Dark Depths has no ice counters on it, sacrifice it. If you do, create a legenda...",44.72
19,Cabal Coffers,Land,"{2}, {T}: Add {B} to your mana pool for each Swamp you control.",29.22
146,Sunken Ruins,Land,"{T}: Add {C} to your mana pool.\n{U/B}, {T}: Add {U}{U}, {U}{B}, or {B}{B} to your mana pool.",20.96
166,"Urborg, Tomb of Yawgmoth",Legendary Land,Each land is a Swamp in addition to its other land types.,19.50
39,Darkslick Shores,Land,Darkslick Shores enters the battlefield tapped unless you control two or fewer other lands.\n{T}: Add {U} or {B} to your mana pool.,16.81
172,Watery Grave,Land — Island Swamp,"({T}: Add {U} or {B} to your mana pool.)\nAs Watery Grave enters the battlefield, you may pay 2 life. If you don't, Watery Grave enters the battlefield tapped.",16.63
25,Command Beacon,Land,"{T}: Add {C} to your mana pool.\n{T}, Sacrifice Command Beacon: Put your commander into your hand from the command zone.",16.29
18,Breeding Pool,Land — Forest Island,"({T}: Add {G} or {U} to your mana pool.)\nAs Breeding Pool enters the battlefield, you may pay 2 life. If you don't, Breeding Pool enters the battlefield tapped.",14.53
119,Reflecting Pool,Land,{T}: Add to your mana pool one mana of any type that a land you control could produce.,10.63
107,Overgrown Tomb,Land — Swamp Forest,"({T}: Add {B} or {G} to your mana pool.)\nAs Overgrown Tomb enters the battlefield, you may pay 2 life. If you don't, Overgrown Tomb enters the battlefield tapped.",9.80


In [24]:
mana_rocks = card_df[card_df['oracle_text'].str.contains('mana pool|Mana pool|{C}{C}', na = False)]
mana_rocks = mana_rocks[-mana_rocks['type_line'].str.contains('Land|land', na = False)]
print('Mana ramp count: ' + str(len(mana_rocks)))
mana_rocks.sort_values('usd', ascending=False)

Mana ramp count: 22


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
12,Bloom Tender,Creature — Elf Druid,2.0,{1}{G},"{T}: For each color among permanents you control, add one mana of that color to your mana pool.",1,1,29.30
127,"Selvala, Heart of the Wilds",Legendary Creature — Elf Scout,3.0,{1}{G}{G},"Whenever another creature enters the battlefield, its controller may draw a card if its power is greater than each other creature's power.\n{G}, {T}: Add X mana in any combination of colors to you...",2,3,20.99
64,Gilded Lotus,Artifact,5.0,{5},{T}: Add three mana of any one color to your mana pool.,NaN,NaN,8.99
11,Birds of Paradise,Creature — Bird,1.0,{G},Flying\n{T}: Add one mana of any color to your mana pool.,0,1,6.95
86,"Kruphix, God of Horizons",Legendary Enchantment Creature — God,5.0,{3}{G}{U},"Indestructible\nAs long as your devotion to green and blue is less than seven, Kruphix isn't a creature.\nYou have no maximum hand size.\nIf unused mana would empty from your mana pool, that mana ...",4,7,6.61
35,Crystalline Crawler,Artifact Creature — Construct,4.0,{4},Converge — Crystalline Crawler enters the battlefield with a +1/+1 counter on it for each color of mana spent to cast it.\nRemove a +1/+1 counter from Crystalline Crawler: Add one mana of any colo...,1,1,4.99
43,Deathrite Shaman,Creature — Elf Shaman,1.0,{B/G},"{T}: Exile target land card from a graveyard. Add one mana of any color to your mana pool.\n{B}, {T}: Exile target instant or sorcery card from a graveyard. Each opponent loses 2 life.\n{G}, {T}: ...",1,2,4.40
158,Thran Dynamo,Artifact,4.0,{4},{T}: Add {C}{C}{C} to your mana pool.,NaN,NaN,3.17
7,Awakening Zone,Enchantment,3.0,{2}{G},"At the beginning of your upkeep, you may create a 0/1 colorless Eldrazi Spawn creature token. It has ""Sacrifice this creature: Add {C} to your mana pool.""",NaN,NaN,3.08
135,Sol Ring,Artifact,1.0,{1},{T}: Add {C}{C} to your mana pool.,NaN,NaN,3.00


In [25]:
card_draw = card_df[card_df['oracle_text'].str.contains('Draw|draw', na = False)]
print('Card draw count: ' + str(len(card_draw)))
card_draw.head() # this one will require more thought

Card draw count: 37


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
16,Braingeyser,Sorcery,2.0,{X}{U}{U},Target player draws X cards.,NaN,NaN,NaN
17,Brainstorm,Instant,1.0,{U},"Draw three cards, then put two cards from your hand on top of your library in any order.",NaN,NaN,0.61
20,Careful Study,Sorcery,1.0,{U},"Draw two cards, then discard two cards.",NaN,NaN,0.59
23,Chart a Course,Sorcery,2.0,{1}{U},Draw two cards. Then discard a card unless you attacked with a creature this turn.,NaN,NaN,0.44
27,Commander's Sphere,Artifact,3.0,{3},{T}: Add to your mana pool one mana of any color in your commander's color identity.\nSacrifice Commander's Sphere: Draw a card.,NaN,NaN,0.25


In [31]:
removal = card_df[card_df['oracle_text'].str.lower().str.contains('destroy target', na = False)]
removal = removal[-removal['type_line'].str.contains('Land')]
print('Targeted removal: ' + str(len(removal)))
removal # this one will require more thought

Targeted removal: 15


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
1,Abrupt Decay,Instant,2.0,{B}{G},Abrupt Decay can't be countered by spells or abilities.\nDestroy target nonland permanent with converted mana cost 3 or less.,NaN,NaN,4.69
2,Acidic Slime,Creature — Ooze,5.0,{3}{G}{G},"Deathtouch (Any amount of damage this deals to a creature is enough to destroy it.)\nWhen Acidic Slime enters the battlefield, destroy target artifact, enchantment, or land.",2,2,0.25
10,Banewhip Punisher,Creature — Human Warrior,3.0,{2}{B},"When Banewhip Punisher enters the battlefield, you may put a -1/-1 counter on target creature.\n{B}, Sacrifice Banewhip Punisher: Destroy target creature that has a -1/-1 counter on it.",2,2,0.09
14,Bone Splinters,Sorcery,1.0,{B},"As an additional cost to cast Bone Splinters, sacrifice a creature.\nDestroy target creature.",NaN,NaN,0.07
22,Caustic Caterpillar,Creature — Insect,1.0,{G},"{1}{G}, Sacrifice Caustic Caterpillar: Destroy target artifact or enchantment.",1,1,0.10
48,Dimir Charm,Instant,2.0,{U}{B},Choose one —\n• Counter target sorcery spell.\n• Destroy target creature with power 2 or less.\n• Look at the top three cards of target player's library. Put one back and the rest into that player...,NaN,NaN,0.09
67,Golgari Charm,Instant,2.0,{B}{G},Choose one —\n• All creatures get -1/-1 until end of turn.\n• Destroy target enchantment.\n• Regenerate each creature you control.,NaN,NaN,0.25
70,"Grimgrin, Corpse-Born",Legendary Creature — Zombie Warrior,5.0,{3}{U}{B},"Grimgrin, Corpse-Born enters the battlefield tapped and doesn't untap during your untap step.\nSacrifice another creature: Untap Grimgrin and put a +1/+1 counter on it.\nWhenever Grimgrin attacks,...",5,5,4.48
74,Hero's Downfall,Instant,3.0,{1}{B}{B},Destroy target creature or planeswalker.,NaN,NaN,2.49
85,Krosan Grip,Instant,3.0,{2}{G},"Split second (As long as this spell is on the stack, players can't cast spells or activate abilities that aren't mana abilities.)\nDestroy target artifact or enchantment.",NaN,NaN,1.49
